In [1]:
import re

In [2]:
class Article:
    def __init__(self, title, media, contents):
        self.title = title
        self.media = media
        self.content = contents

    def readContent(self):
        for line in self.content:
            if line is '': continue
            yield line            

In [3]:
class ArticleReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:-](?:["\']|(?![0-9])))')
        self.content = ''

    def __iter__(self):
        def is_splited_sentence(sentences):
            return len(sentences) > 1
        
        with open(self.filepath, encoding='utf-8') as f:
            title = f.readline()[:-1]
            yield title
            self.content = f.readline()[:-1]

            media = f.readline()[:-1]
            yield media

            self.del_personal_info(media)
            docs = self.rgxSplitter.split(self.content)
            
            if not is_splited_sentence(docs): # 본문이 1줄이며, 위 정규식에 따라 split 되지 않음
                yield docs[0]
            else :
                for s in map(lambda a, b: a + b, docs[::2], docs[1::2]):
                    if not s: continue
                    yield s

    def del_personal_info(self, media):
        rmBracket = re.sub('(\([^)]*\)|\[[^]]*\])', '', self.content)  # 괄호 안 내용 제거
        rmMedia = re.sub(media, ' ', rmBracket)  # 언론사명 제거
        rmReporter = re.sub('[가-힣]{2,5}\s?기자', ' ', rmMedia) # 기자 이름 제거
        rmEmail = re.sub('[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*@[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*\.[a-zA-Z]{2,3}', ' ', rmReporter) # 이메일 제거

        self.content = rmEmail
